In [24]:
!pip install transformers torch
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd
import re

In [26]:
dir_raw_id = 62
dir_process_id = 63

In [27]:
datanode = DataNode()

Init DataNode sucessful!


In [28]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='eventseye.json'))
meta

{'owner': 'nutnicha',
 'human_size': '58.12 KB',
 'name': 'eventseye.json',
 'description': 'file eventseye.json',
 'directory': 62,
 'path': 'nutnicha/DSI310-08/rawData/eventseye.json'}

In [29]:
data = json.load(fp)
data

[{'Exhibition_Name': '',
  'Describe': '',
  'Country': '',
  'City': '',
  'Place': ''},
 {'Exhibition_Name': 'BRIDAL ASIA - NEW DELHI',
  'Describe': 'Wedding Show. Bridal Asia brings to you a first-of-its kind exhibition with a handpicked collection of bridal couture, jewellery & accessories',
  'Country': 'India',
  'City': 'New Delhi ',
  'Place': 'JW Marriott Hotel New Delhi Aerocity'},
 {'Exhibition_Name': 'SALON DU MARIAGE - LYON',
  'Describe': 'Lyon Wedding Fair. New trends, the best providers in your region. Wedding dresses, costumes, jewelers, announcements, decoration, florists, wedding planner, hairstyles, accessories, caterers, photographers, videographers...',
  'Country': 'France',
  'City': 'Lyon ',
  'Place': 'Eurexpo'},
 {'Exhibition_Name': 'SALON TENDANCE MARIAGE DE LYON',
  'Describe': 'Lyon wedding fair. During this weekend, more than sixty professionals from the region will be at your disposal to make the day of your dreams come true. Three daily parades, weddin

In [30]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [31]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [33]:
example = "LUXURY BRIDAL EXPO GEORGIOS BANQUETS ORLAND PARK"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-ORG', 'score': 0.87962085, 'index': 9, 'word': '▁GE', 'start': 18, 'end': 21}, {'entity': 'I-ORG', 'score': 0.6070428, 'index': 10, 'word': 'ORG', 'start': 21, 'end': 24}, {'entity': 'I-ORG', 'score': 0.71335226, 'index': 11, 'word': 'IOS', 'start': 24, 'end': 27}, {'entity': 'B-LOC', 'score': 0.50927746, 'index': 15, 'word': '▁OR', 'start': 36, 'end': 39}, {'entity': 'I-LOC', 'score': 0.59207296, 'index': 16, 'word': 'LAND', 'start': 39, 'end': 43}, {'entity': 'I-LOC', 'score': 0.48832163, 'index': 17, 'word': '▁PAR', 'start': 43, 'end': 47}]


In [34]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [42]:
text = data[5]['Country']
text

'UAE - United Arab Emirates'

In [43]:
output = nlp(text)
output

[{'entity': 'B-LOC',
  'score': 0.99958235,
  'index': 1,
  'word': '▁U',
  'start': 0,
  'end': 1},
 {'entity': 'I-LOC',
  'score': 0.9990067,
  'index': 2,
  'word': 'AE',
  'start': 1,
  'end': 3},
 {'entity': 'B-LOC',
  'score': 0.7131397,
  'index': 4,
  'word': '▁United',
  'start': 5,
  'end': 12},
 {'entity': 'I-LOC',
  'score': 0.93449163,
  'index': 5,
  'word': '▁Arab',
  'start': 12,
  'end': 17},
 {'entity': 'I-LOC',
  'score': 0.9489847,
  'index': 6,
  'word': '▁Emirat',
  'start': 17,
  'end': 24},
 {'entity': 'I-LOC',
  'score': 0.9679328,
  'index': 7,
  'word': 'es',
  'start': 24,
  'end': 26}]

In [44]:
post_process(output)

[{'type': 'I-LOC', 'text': ' UAE'},
 {'type': 'I-LOC', 'text': ' United Arab Emirates'},
 {'type': 'I-LOC', 'text': ' Arab Emirates'},
 {'type': 'I-LOC', 'text': ' Emirates'}]

In [45]:
#Process news to spo

In [46]:
spo = []
for eventseye in tqdm(data):
    Exhibition_Name = eventseye.get('Exhibition_Name') 
    if Exhibition_Name == None:
        continue
    ner = nlp(Exhibition_Name)
    processed = post_process(ner)
    for kw in processed:
        spo.append({
            'Exhibition_Name': eventseye.get('Exhibition_Name', ""),
            'Describe': eventseye.get('Describe', ""),
            'Country': eventseye.get('Country', ""),
            'City': eventseye.get('City', ""),
            'Place': eventseye.get('Place', ""),
        })

  0%|          | 0/186 [00:00<?, ?it/s]

In [47]:
df = pd.DataFrame(spo)
df['Exhibition_Name'] = df['Exhibition_Name'].apply(lambda x: x[2:])
df

,Exhibition_Name,Describe,Country,City,Place
0,IDAL ASIA - NEW DELHI,Wedding Show. Bridal Asia brings to you a firs...,India,New Delhi,JW Marriott Hotel New Delhi Aerocity
1,IDAL ASIA - NEW DELHI,Wedding Show. Bridal Asia brings to you a firs...,India,New Delhi,JW Marriott Hotel New Delhi Aerocity
2,IDAL ASIA - NEW DELHI,Wedding Show. Bridal Asia brings to you a firs...,India,New Delhi,JW Marriott Hotel New Delhi Aerocity
3,IDAL ASIA - NEW DELHI,Wedding Show. Bridal Asia brings to you a firs...,India,New Delhi,JW Marriott Hotel New Delhi Aerocity
4,LON DU MARIAGE - LYON,"Lyon Wedding Fair. New trends, the best provid...",France,Lyon,Eurexpo
...,...,...,...,...,...
232,NG KONG JEWELLERY & GEM FAIR,Jewellery & Gem Fair. June Hong Kong Jewellery...,Hong Kong,Hong Kong,Hong Kong Convention & Exhibition Centre
233,NG KONG JEWELLERY & GEM FAIR,Jewellery & Gem Fair. June Hong Kong Jewellery...,Hong Kong,Hong Kong,Hong Kong Convention & Exhibition Centre
234,W DESIGNERS,The UK’s most important exhibition for emergin...,UK - United Kingdom,London,Business Design Centre Islington
235,W DESIGNERS,The UK’s most important exhibition for emergin...,UK - United Kingdom,London,Business Design Centre Islington


In [48]:
#write DataNode to DataPlatform

In [49]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='eventseye.json')])

{'sucess': True,
 'file_id': 200,
 'path': 'nutnicha/DSI310-08/processData/spo.parquet',
 'lineage': True}